In [1]:
%load_ext autoreload
%autoreload 2

import numpy as np
import napari
from PIL import Image

from scribbles_creator import *
from cellpose_data_handler import *

c:\Users\roman\anaconda3\envs\ilastik2\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Prediction

Define prediction parameters

In [8]:
# Where to find and save the data
# folder_path = "./cellpose_results/data/run02_examples"
folder_path = "./cellpose_train_imgs"

mode = "all"
bins = [0.1]#, 0.025, 0.05, 0.1, 0.25, 0.5, 1]
scribble_width = 2
# all_suff = ["a", "b", "c", "d", "e", "f", "g", "h", "i", "j"]
# suff = all_suff[:1]
s = "a"

# Define the convpaint parameters
convpaint_layers = [[0], [0,2], [0,2,5]] # layers to use for convpaint (depending on the model chosen)
convpaint_scalings = [[1], [1,2], [1,2,4], [1,2,4,8]] # scalings for convpaint (downscaling the image)
model="vgg16" # 'vgg16', 'efficient_netb0', 'single_layer_vgg16', 'single_layer_vgg16_rgb', 'dino_vits16'
use_dino = True
use_ilastik = True
pred_seed = 123 # seed used for random forest classifier

save_res = False
show_res = True

In [21]:
for bin in [0.1]: #bins:
    for img_num in [0]:#range(0, 540, 10):#range(0, 10):
        print(f"IMG {img_num}: bin {bin}, suff {s}")
        for layers in []:#[[0,2,5]]:#convpaint_layers:
            for scalings in [[1,2]]:#convpaint_scalings:
                print(f"   ConvPaint: layers {layers}, scalings {scalings}")
                pred_conv = pred_cellpose_convpaint(folder_path, img_num, mode=mode, bin=bin, scribble_width=scribble_width, suff=s,
                                                    layer_list=layers, scalings=scalings, model=model, random_state=pred_seed,
                                                    save_res=save_res, show_res=show_res)
        if use_ilastik:
            print("   Ilastik")
            pred_ila = pred_cellpose_ilastik(folder_path, img_num, mode=mode, bin=bin, scribble_width=scribble_width, suff=s,
                                                 random_state=pred_seed,
                                                 save_res=save_res, show_res=show_res)
        if use_dino:
            print("   DINO")
            pred_dino = pred_cellpose_dino(folder_path, img_num, mode=mode, bin=bin, scribble_width=scribble_width, suff=s,
                                           dinov2_model='s', dinov2_layers=(), dinov2_scales=(), upscale_order=0, random_state=pred_seed,
                                           save_res=save_res, show_res=show_res)
        print("\n")

IMG 0: bin 0.1, suff a
   DINO
Active channels: R=True, G=True, B=False --> Removed 1 channel(s) --> shape: (2, 383, 512)
(2, 383, 512) (383, 512)




Show the last predictions (optionally with image, ground truth and scribbles)

In [ ]:
img_data = get_cellpose_img_data(folder_path, img_num, load_img=True, load_gt=True, load_scribbles=True, mode=mode, bin=bin, scribble_width=scribble_width, suff=s)
img = img_data["img"]
ground_truth = img_data["gt"]
scribbles = img_data["scribbles"]

In [ ]:
np.sum(pred_conv == ground_truth) / np.prod(pred_conv.shape)

In [ ]:
v = napari.Viewer()
v.add_image(img)
v.add_labels(ground_truth)
v.add_labels(scribbles)
v.add_labels(pred_conv)
v.add_labels(pred_ila)
v.add_labels(pred_dino)